In [ ]:
pip install streamlit openai mem0ai agno

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.7/868.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 12.3 MB/s eta 0:00:00


In [ ]:
import os
from agno.agent import Agent
from agno.models.openai.like import OpenAILike
import google.generativeai as genai
from mem0 import MemoryClient

Configure API (Gemini, Mem0, and Sutra-v2)

In [ ]:
genai.configure(api_key="YOUR_GEMINI__API_KEY")
gemini = genai.GenerativeModel("gemini-1.5-flash")

# Initialize Mem0 Agent
client = MemoryClient(api_key="YOUR_MEM0_API_KEY")

# Initialize SUTRA Agent via agno
sutra_agent = Agent(
    model=OpenAILike(
        id="sutra-v2",
        api_key="YOUR_SUTRA-v2_API_KEY",
        base_url="https://api.two.ai/v2"
    ),
    markdown=True,
    description="A multilingual medical assistant powered by SUTRA-V2",
    instructions=["Answer concisely in the requested language."]
)


Colab Based Chatbot (Enhanced version for custom user_id retrival)

In [ ]:
# === User Login ===
user_id = input("Enter your unique username or ID: ")
search_results = client.search("check", user_id=user_id)

def process_input(question):
    answer = input(question)
    detect_prompt = f"Detect language of this: {answer} (only 'en' or 'mr')"
    response = sutra_agent.run(detect_prompt)
    lang = response.content.strip()
    if lang == 'mr':
        translate_prompt = f"Translate this Marathi to English: {answer}"
        response = sutra_agent.run(translate_prompt)
        return response.content.strip()
    else:
        return answer

# === New User: Collect basic info ===
if len(search_results) == 0:
    print("New user detected. Collecting basic info...")

    name = process_input("What's your full name? / तुमचं पूर्ण नाव काय आहे? : ")
    age = process_input("What is your age? / तुमचं वय किती आहे? : ")
    diabetes_type = process_input("What type of Diabetes? / डायबिटीजचा प्रकार? : ")
    medication = process_input("Any medication? / कोणती औषधे घेतलीत? : ")
    symptoms = process_input("Unusual symptoms? / काही लक्षणे? : ")
    current_location = proccess_input("Where are you currently living? / तुम्ही सध्या कुठे राहता?")

    messages = [
        {"role": "user", "content": f"My name is {name}."},
        {"role": "user", "content": f"I am {age} years old."},
        {"role": "user", "content": f"I have {diabetes_type} diabetes."},
        {"role": "user", "content": f"My medications include: {medication}."},
        {"role": "user", "content": f"My symptoms include: {symptoms}."},
        {"role": "user", "content": f"I currentlyl live in: {current_location}."}
        {"role": "assistant", "content": f"Thanks {name}, your health info is stored for personalized support."}
    ]
    client.add(messages, user_id=user_id)
    print("✅ Basic info saved.")
else:
    print("✅ Existing user detected. Memories retrieved.")

# === Chat Loop ===
print("\nDiabetes Chatbot Ready! (type 'exit' to quit)\n")

while True:
    query = input("You: ")
    if query.lower() == 'exit':
        print("Chat ended. Stay healthy!")
        break

    # Detect language
    detect_prompt = f"Detect language: {query} (only 'en' or 'mr')"
    response = sutra_agent.run(detect_prompt)
    detected_lang = response.content.strip()

    # Translate Marathi to English if needed
    if detected_lang == 'mr':
        translate_prompt = f"Translate to English: {query}"
        response = sutra_agent.run(translate_prompt)
        query_en = response.content
    else:
        query_en = query

    # Retrieve Mem0 context
    search_results = client.search(query_en, user_id=user_id)

    # Collect context from both 'message' and 'memory'
    mem0_memories = []
    for r in search_results:
        if 'message' in r and 'content' in r['message']:
            role = r['message'].get('role', 'user').capitalize()
            content = r['message']['content']
            mem0_memories.append(f"{role}: {content}")
        elif 'memory' in r:
            mem0_memories.append(f"Memory: {r['memory']}")

    context = "\n".join(mem0_memories)

    # Extract Personal Info
    user_name = "User"
    diabetes_type = "Not specified"
    location = "Not specified"

    for r in search_results:
        memory_text = ''
        if 'message' in r and 'content' in r['message']:
            memory_text = r['message']['content'].lower()
        elif 'memory' in r:
            memory_text = r['memory'].lower()

        if memory_text:
            if "name is" in memory_text:
                user_name = memory_text.split("name is")[-1].split()[0].capitalize()
            if "type 1" in memory_text or "type 2" in memory_text:
                diabetes_type = "Type 2" if "type 2" in memory_text else "Type 1"
            if "from" in memory_text:
                location = memory_text.split("from")[-1].split(".")[0].strip().capitalize()

    # === Conditional Prompt Based on Detected Language ===
    if detected_lang == 'mr':
        final_prompt = f"""
    तुम्ही एक मधुमेह सहाय्यक आहात. {user_name} नावाच्या रुग्णाला {location} येथील {diabetes_type} मधुमेह असलेल्या रुग्णास मदत करत आहात.

    पूर्वीच्या संभाषणातील आठवणी:
    {context}

    वापरकर्त्याचा नवीन प्रश्न:
    "{query_en}"

    भारतासाठी उपयुक्त, मधुमेह-सुरक्षित आणि व्यावहारिक उत्तर द्या.
    """
    else:
        final_prompt = f"""
    You are a diabetes-friendly AI assistant helping {user_name}, a patient from {location} diagnosed with {diabetes_type} Diabetes.

    Past memories from previous chats:
    {context}

    User's new query:
    "{query_en}"

    Provide a helpful, India-specific, diabetes-safe, and practical response.
    """

    # === Gemini generates response ===
    response = gemini.generate_content(final_prompt)

    # === Translate output to Marathi if needed ===
    if detected_lang == 'mr':
        translate_back_prompt = f"Translate this to Marathi: {response.text}"
        response_back = sutra_agent.run(translate_back_prompt)
        output = response_back.content
        print("\nBot (Marathi):", output)
    else:
        print("\nBot (English):", response.text)

Enter your unique username or ID: Rohan
✅ Existing user detected. Memories retrieved.

Diabetes Chatbot Ready! (type 'exit' to quit)

You: what is my diabeties type ?

Bot (English): Hello Rohan Singh,

Based on our previous conversations, you have Type 1 Diabetes.  It's important to remember that Type 1 diabetes is an autoimmune condition where your body doesn't produce insulin.  Metformin, which you are currently taking, is typically used for Type 2 diabetes, and its use in Type 1 diabetes is usually not recommended unless in specific circumstances.  

Since you're currently taking Metformin, I strongly advise you to discuss this medication with your doctor or endocrinologist.  They can confirm the correct diagnosis and ensure your treatment plan is appropriate for your specific needs and Type 1 Diabetes.  They can also address any questions or concerns you have about your medication.

In India, you have many resources available to manage your diabetes effectively.  Remember to follo